In [1]:
import src.data as data

data.DataFetcher.main()
data.PreProcess.main(False)

Update Files
Mon Apr  1 10:38:36 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.08 Secs
Mon Apr  1 10:38:36 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.06 Secs
Mon Apr  1 10:38:36 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Mon Apr  1 10:38:39 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 3.36 Secs
Mon Apr  1 10:38:47 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 7.69 Secs
Mon Apr  1 10:38:49 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240328.feather Done! Cost 0.31 Secs
Mon Apr  1 10:38:49 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240329.feather Done! Cost 0.18 Secs
Mon Apr  1 10:38:49 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240329.feather Done! Cost 0.22 Secs
Mon Apr  1 10:38:52 2024 : Updated ~ data\DataBase\DB_trade\day\2024\day.20240329.feather Done! 

24-04-01 10:39:14|MOD:PreProcess  |: if_train is False , Data Processing start!
24-04-01 10:39:14|MOD:PreProcess  |: 3 datas :['y', 'trade_day', 'trade_30m']


Mon Apr  1 10:39:14 2024 : All Updates Done! Cost 37.99 Secs
Mon Apr  1 10:39:14 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.64 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.57 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 2.79 secs
models blocks merging ...... cost 0.21 secs
y blocks process ...... cost 2.82 secs
y blocks masking ...... cost 0.12 secs
y blocks saving  ...... cost 0.12 secs
y blocks norming ...... cost 0.00 secs
Mon Apr  1 10:39:21 2024 : y finished! Cost 7.34 Seconds
Mon Apr  1 10:39:21 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.87 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.05 secs
trade_day blocks masking ...... cost 0.07 secs
trade_day blocks saving  ...... cost 0.10 secs
trade_day blocks norming ...... cost 0.00 secs
Mon Apr  1 10:39:22 2024 : trade_day finished! Cost 1.10 Seconds


24-04-01 10:39:28|MOD:PreProcess  |: Data Processing Finished! Cost 14.23 Seconds


... cost 0.87 secs
trade_30m blocks norming ...... cost 0.00 secs
Mon Apr  1 10:39:28 2024 : trade_30m finished! Cost 5.53 Seconds


In [ ]:
from run_model import predict
from src.func import date as date
df = predict('gru_day' , model_num = 0)
df[df['pred_date'] >= date.today(-10)].pivot_table(values = 'gru_day' , index = 'secid' , columns = 'date').fillna(0).corr()

In [17]:
i

20170104

In [10]:
import os
import pandas as pd

from dataclasses import dataclass , field   
from typing import Any , Literal , ClassVar

@dataclass
class DeployModelPred:
    model_name : str
    model_type : Literal['best' , 'swalast' , 'swabest'] = 'swalast'
    model_num  : int = 0
    alias : str | None = None

    destination : ClassVar[str] = '//hfm-pubshare/HFM各部门共享/量化投资部/龙昌伦/Alpha'

    def __post_init__(self):
        if self.alias is None: self.alias = self.model_name

    def deploy(self , df : pd.DataFrame , secid_col = 'secid' , date_col = 'date'):
        os.makedirs(f'{self.destination}/{self.alias}' , exist_ok=True)
        for date , subdf in df.groupby(date_col):
            path = f'{self.destination}/{self.alias}/{self.alias}_{date}.txt'
            subdf.drop(columns='date').set_index(secid_col).to_csv(path, sep='\t', index=True, header=False)


deploy_list = [
    DeployModelPred('gru_day' , 'swalast' , 0 , 'gru_day_V0')
]


In [11]:
import pandas as pd
df = pd.read_feather('gru_day_preds.feather')
deploy_list[0].deploy(df)

In [12]:
df

,secid,date,gru_day
0,1,20170104,-0.676739
1,2,20170104,0.246266
2,4,20170104,0.765674
3,5,20170104,-1.278355
4,6,20170104,-3.938847
...,...,...,...
5088,688799,20240313,2.169616
5089,688800,20240313,1.703681
5090,688819,20240313,0.466089
5091,688981,20240313,1.229148
